In [2]:
import pandas as pd
import numpy as np
import math 
import scipy.optimize as opt
import pipeline as pxy

In [3]:
dfs = pxy.prepare_data("processed_noed.csv")

In [4]:
df, mdf, wdf = dfs

In [5]:
# I'll compare two age groups: pre- and during college, and post-college.

In [6]:
df1 = df.loc[df.age_ms <=21,:].copy()
mdf1 = mdf.loc[mdf.age_ms <=21,:].copy()
wdf1 = wdf.loc[wdf.age_ms <=21,:].copy()
df2 = df.loc[df.age_ms >21,:].copy()
mdf2 = mdf.loc[mdf.age_ms >21,:].copy()
wdf2 = wdf.loc[wdf.age_ms >21,:].copy()

In [7]:
len(df2)

833

In [55]:
values_wanted = [len(df), 
                 len(mdf), 
                 len(wdf), 
                 df.married_status.value_counts()[1],
                 np.mean(df.age_ms),
                 np.mean(df.wage_ms),
                 np.mean(df.t_single),
                 np.mean(df.t_married)
                ]

values_wanted2 = [len(df1), 
                 len(mdf1), 
                 len(wdf1), 
                 df1.married_status.value_counts()[1],
                 np.mean(df1.age_ms),
                 np.mean(df1.wage_ms),
                 np.mean(df1.t_single),
                 np.mean(df1.t_married)
                ]

values_wanted3 = [len(df2), 
                 len(mdf2), 
                 len(wdf2), 
                 df2.married_status.value_counts()[1],
                 np.mean(df2.age_ms),
                 np.mean(df2.wage_ms),
                 np.mean(df2.t_single),
                 np.mean(df2.t_married)
                ]

In [58]:
sumDF = pd.DataFrame({"Variable": ["No. of Persons", 
                                  "No. of Men",
                                  "No. of Women",
                                  "No. of Married",
                                  "Mean Age Married",
                                 "Mean Wage at Marriage",
                                 "Mean single-time",
                                 "Mean married-time"],
                    "Total": values_wanted,
                     "Married While Young": values_wanted2,
                     "Married While Older": values_wanted3})
print(sumDF.round(1).to_latex(index = False))

\begin{tabular}{lrrr}
\toprule
              Variable &   Total &  Married While Young &  Married While Older \\
\midrule
        No. of Persons &  1400.0 &                567.0 &                833.0 \\
            No. of Men &   647.0 &                200.0 &                447.0 \\
          No. of Women &   753.0 &                367.0 &                386.0 \\
        No. of Married &   931.0 &                345.0 &                586.0 \\
      Mean Age Married &    24.0 &                 19.5 &                 27.0 \\
 Mean Wage at Marriage &     3.7 &                  2.6 &                  4.4 \\
      Mean single-time &     9.0 &                  4.5 &                 12.0 \\
     Mean married-time &    19.1 &                 18.1 &                 19.8 \\
\bottomrule
\end{tabular}



# Summary Statistics:

I want the following: 
1. Count 

In [10]:
def prepare_data(fn):
    # reads a fn
    # outputs a tuple containing df, mdf, and wdf
    df = pd.read_csv(fn)
    df["married_status"] = 0
    df.loc[df.year_ended > 1993, "married_status"] = 1.0
    df["t_single"] = df["age_ms"] - 15
    df["t_married"] = df["age_me"] -  df["age_ms"]    

    df.loc[df.t_married > 25,"t_married"]= 25

    df = df.loc[df.t_married >0,:].copy()
    df['z_raw'] = np.exp(df.wage_ms)
    # df["z"] = pd.qcut(df.z_raw, q = 7, duplicates= "drop", labels =[1,2,3,4,5,6,7])
    mdf = df.loc[df.gender ==1, :].copy().reset_index()
    wdf = df.loc[df.gender ==0, :].copy().reset_index()
    mdf["z"], bins_mz = pd.qcut(mdf.z_raw, q = 5, labels =[1,2,3,4,5], retbins=True)
    wdf["z"], bins_wz = pd.qcut(wdf.z_raw, q = 5, labels =[1,2,3,4,5], retbins=True)

    mdf["x"] = np.nan
    mdf.loc[mdf.z == 1, "x"] = np.median(mdf.loc[mdf.z==1, "z_raw"])
    mdf.loc[mdf.z == 2, "x"] = np.median(mdf.loc[mdf.z==2, "z_raw"])
    mdf.loc[mdf.z == 3, "x"] = np.median(mdf.loc[mdf.z==3, "z_raw"])
    mdf.loc[mdf.z == 4, "x"] = np.median(mdf.loc[mdf.z==4, "z_raw"])
    mdf.loc[mdf.z == 5, "x"] = np.median(mdf.loc[mdf.z==5, "z_raw"])

    wdf["x"] = np.nan
    wdf.loc[wdf.z == 1, "x"] = np.median(wdf.loc[wdf.z==1, "z_raw"])
    wdf.loc[wdf.z == 2, "x"] = np.median(wdf.loc[wdf.z==2, "z_raw"])
    wdf.loc[wdf.z == 3, "x"] = np.median(wdf.loc[wdf.z==3, "z_raw"])
    wdf.loc[wdf.z == 4, "x"] = np.median(wdf.loc[wdf.z==4, "z_raw"])
    wdf.loc[wdf.z == 5, "x"] = np.median(wdf.loc[wdf.z==5, "z_raw"])

    # assign partner's z & x

    partnerDF = pd.DataFrame({"i_type": list(wdf["z"].drop_duplicates().sort_values()),
                             "x": list(wdf["x"].drop_duplicates().sort_values())})
    for index, row in partnerDF.iterrows():
        lb = bins_wz[index]
        ub = bins_wz[index + 1]
        if index == 0:
            lb = -0.01
        partnerDF.loc[index, "lb"] = lb
        partnerDF.loc[index, "ub"] = ub

    mdf["partner_z"] = np.nan
    mdf["partner_x"] = np.nan

    for index, row in mdf.iterrows():
        temp_wage = row["partnerwage_ms"]
        temp_z = np.exp(temp_wage)
        temp = partnerDF.loc[(partnerDF.lb < temp_z) & (partnerDF.ub >= temp_z),["i_type", 'x']]
        try:
            mdf.loc[index, "partner_z"] = np.float(temp.i_type)
        except:
            pass
        try:
            mdf.loc[index, "partner_x"] = np.float(temp.x)
        except: 
            pass


    return (df, mdf, wdf)

def calc_mrhs(wdf, lb, ub):
    acceptance_set = [i for i in range(1,6) if (i >= lb) & (i <= ub)]
    big_sum = 0
    ri = np.median(wdf.loc[wdf.z == lb, "x"])
    for xj in acceptance_set:
        big_sum = big_sum + 0.2*((np.median(wdf.loc[wdf.z == xj, "x"]))-ri)
    rhs = 2 + (est_lambda)/(est_beta + est_delta)*big_sum
    return(rhs)

def find_reserve(df_opp, ub, params):
    est_lambda, est_delta = params
    est_beta = 0.05

    diff_df = pd.DataFrame({'r' : [i for i in range(1,ub + 1)],
        "lhs": np.nan,
        "rhs": np.nan,
        "delta": np.nan})

    for i in range(1,ub + 1):
        lb = ub + 1-i
        acceptance_set = [i for i in range(1,6) if (i >= lb) & (i <= ub)]
        big_sum = 0
        ri = np.median(df_opp.loc[df_opp.z == lb, "x"])
        diff_df.loc[diff_df.r == lb, "lhs"] = ri 
        for xj in acceptance_set:
            big_sum = big_sum + 0.2*((np.median(df_opp.loc[df_opp.z == xj, "x"]))-ri)
        rhs = 2 + (est_lambda)/(est_beta + est_delta)*big_sum
        diff_df.loc[diff_df.r == lb, "rhs"] = rhs 
        delta = ri-rhs
        diff_df.loc[diff_df.r == lb, "delta"] = delta 

    diff_df["abs_delta"] = np.abs(diff_df.delta)
    chosen_type = diff_df.sort_values("abs_delta").reset_index(drop = True).loc[0,"r"]
    # chosen_type = diff_df.loc[diff_df.delta >0, :].sort_values("delta").reset_index(drop = True).loc[0,"r"]
    return(chosen_type)

def set_max_alt(df_opp):
    
    try:
        ub = df_opp.loc[df_opp.reserv.isnull(),:].sort_values("type", ascending = False).reset_index(drop = True).loc[0, "type"]
    except:
        ub = 1
        pass
    return(ub)

def reserve_alt(df_main, df_opp, main_type, params):
    # set max of
    ub = df_opp.loc[df_opp.reserv.isnull(),:].sort_values("type", ascending = False).reset_index(drop = True).loc[0, "type"]
    # retrieve highest empty spot, that will be new max
    diff_df = pd.DataFrame({'r' : [i for i in range(1,ub + 1)],
            "lhs": np.nan,
            "rhs": np.nan,
            "delta": np.nan,})
    for i in range(1,ub + 1):
        lb = ub + 1-i
        acceptance_set = [i for i in range(1,6) if (i >= lb) & (i <= ub)]
        big_sum = 0
        ri = np.median(df_opp.loc[df_opp.z == lb, "x"])
        diff_df.loc[diff_df.r == lb, "lhs"] = ri 
        for xj in acceptance_set:
            big_sum = big_sum + 0.2*((np.median(df_opp.loc[df_opp.z == xj, "x"]))-ri)
        rhs = 2 + (est_lambda)/(est_beta + est_delta)*big_sum
        diff_df.loc[diff_df.r == lb, "rhs"] = rhs 
        delta = ri-rhs
        diff_df.loc[diff_df.r == lb, "delta"] = delta 
    diff_df["abs_delta"] = np.abs(diff_df.delta)
    chosen_type = diff_df.sort_values("abs_delta").reset_index(drop = True).loc[0,"r"]
    return(chosen_type)

In [11]:
def generate_matchsets(mdf, wdf, params):
    est_lambda, est_delta = params
    est_beta = 0.05

    menDF= pd.DataFrame({'type' : [6-i for i in range(1,6)],
                         "max": np.nan,
                         "reserv": np.nan})
    womenDF = pd.DataFrame({'type' : [6-i for i in range(1,6)],
                         "max": np.nan,
                                  "reserv": np.nan})

    # set max for top categories of men and women
    menDF.loc[menDF.type == 5, "max"] = 5
    womenDF.loc[womenDF.type == 5, "max"] = 5
    menDF.loc[menDF.type == 5, "reserv"] = find_reserve(wdf, 5, params)
    womenDF.loc[womenDF.type == 5, "reserv"] = find_reserve(mdf, 5, params)

    # for men
    for i in range(1,5):
        i_type = 5-i
        
        man_max_exists = False
        
        # set max for men if identifiable
        try:
            max_men_i = list(womenDF.loc[womenDF.reserv <= i_type, "type"].sort_values(ascending = False))[0]
            man_max_exists = True

        except:
            pass
        
        # set reserve for men if identifiable 
        if man_max_exists == True:
            # set max:
            menDF.loc[menDF.type == i_type, "max"] = max_men_i
            #print("Max of men of type {} = {}".format(i_type, max_men_i))
            menDF.loc[menDF.type == i_type, "reserv"] = find_reserve(wdf, max_men_i, params)
            #print("Reserve of men of type {} = {}".format(i_type, find_reserve(wdf, max_men_i, est_params)))
  
        # set max for max if unidentifable:
        
        if man_max_exists == False:
            new_man_max = set_max_alt(womenDF)
            menDF.loc[menDF.type ==i_type, "max"] = new_man_max
            #print("Max of men of type {} = {}".format(i_type, new_man_max))
            menDF.loc[menDF.type == i_type, "reserv"] = find_reserve(wdf, new_man_max, params)
            #print("Reserve of men of type {} = {}".format(i_type, find_reserve(wdf, new_man_max, est_params)))


        # set max for women if identifiable
        woman_max_exists = False
        
        try:
                max_women_i = list(menDF.loc[menDF.reserv <= i_type, "type"].sort_values(ascending = False))[0]
                woman_max_exists = True
        except:
            pass

        if woman_max_exists == True:
            # set max:
            womenDF.loc[womenDF.type == i_type, "max"] = max_women_i
            #print("Max of women of type {} = {}".format(i_type, max_women_i))
            womenDF.loc[womenDF.type == i_type, "reserv"] = find_reserve(mdf, max_women_i, params)
            #print("Reserve of women of type {} = {}".format(i_type, find_reserve(mdf, max_women_i, est_params)))
        
        if woman_max_exists == False:
            new_woman_max = set_max_alt(menDF)
            womenDF.loc[womenDF.type ==i_type, "max"] = new_woman_max
            #print("Max of women of type {} = {}".format(i_type, new_woman_max))
            womenDF.loc[womenDF.type == i_type, "reserv"] = find_reserve(mdf, new_woman_max, params)
            #print("Reserve of women of type {} = {}".format(i_type, find_reserve(mdf, new_woman_max, est_params)))

    # go back to revise maxes:

    for i in range(1,6):
        i_type = 6-i
        # check for men:
        real_max = max(womenDF.loc[womenDF.reserv <= i_type, "type"].sort_values(ascending = False))
        actual_max = np.float(menDF.loc[menDF.type == i_type, "max"])
        if real_max > actual_max:
            menDF.loc[menDF.type == i_type, "max"] = real_max
        
        # check for women:
        try:
            real_max = max(menDF.loc[menDF.reserv <= i_type, "type"].sort_values(ascending = False))
            actual_max = np.float(womenDF.loc[womenDF.type == i_type, "max"])
            if real_max > actual_max:
                womenDF.loc[womenDF.type == i_type, "max"] = real_max
        except: 
            pass
    return (menDF, womenDF)

In [12]:
def generate_likelihoods(df, mdf, wdf, params):
    # unpack parameters
    est_lambda, est_delta = params
    est_beta = 0.05

    match_sets = generate_matchsets(mdf, wdf, params)
    match_men, match_women =  match_sets[0], match_sets[1]

    mdf["lik_single"] = np.nan
    mdf["lik_married"] = np.nan

    for index, row in mdf.iterrows():
        # likelihood of being single:
        temp_z = row.z
        temp_max = np.float(match_men.loc[match_men.type == row.z, "max"])
        temp_reserv = np.float(match_men.loc[match_men.type == row.z, "reserv"])
        single_time = row.t_single
        married_time = row.t_married
        temp_gamma = 0.2 * (1 + temp_max - temp_reserv)
        likelihood_single = est_lambda * temp_gamma * np.exp(-est_lambda*temp_gamma*single_time)

        # likelihood of marriage
        fxj = 1/(1 + temp_max - temp_reserv)
        likelihood_married = fxj * est_delta * np.exp(-est_delta * married_time)
        mdf.loc[index, "lik_single"] = likelihood_single
        mdf.loc[index, "lik_married"] = likelihood_married

    mdf["likelihood"] =  mdf["lik_single"] * mdf["lik_married"] 
    return mdf["likelihood"]

def criterion_function(params, args):
    df = args[0]
    mdf = args[1]
    wdf = args[2]
    likelihood_vector = generate_likelihoods(df, mdf, wdf, params)
    negloglik = -sum(np.log(likelihood_vector))
    return negloglik


In [13]:
params_initial = np.array([0.05, 0.05])
mle_args1 = [df1, mdf1, wdf1]
results1 = opt.minimize(criterion_function, 
                       params_initial, 
                       args=(mle_args1), 
                       method='L-BFGS-B', 
                       bounds=((0.01, 3), (0.01, 3)),
                       tol=1e-9,
                       options={'maxiter':1000})
results1

      fun: 1363.8820436700698
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00011369, 0.00065938])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 51
      nit: 13
   status: 0
  success: True
        x: array([0.75872514, 0.05319149])

In [14]:
lambda_mle1, delta_mle1 = results1.x

In [15]:
vcv_mle1 = results1.hess_inv.todense()
se_lambda1 = np.sqrt(vcv_mle1[0,0])
se_delta1 = np.sqrt(vcv_mle1[1,1])

In [16]:
print("Lambda_hat is {}, with a standard error of {}.\n".format(lambda_mle1, se_lambda1))
print("Delta_hat is {}, with a standard error of {}.\n".format(delta_mle1, se_delta1))

Lambda_hat is 0.7587251420941598, with a standard error of 0.10599913693477789.

Delta_hat is 0.053191492063554856, with a standard error of 0.011108918373657172.



In [17]:
mle_men1, mle_women1 = generate_matchsets(mdf1, wdf1, [lambda_mle1, delta_mle1])

In [18]:
print(mle_men1.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     5.0 \\
    4 &  4.0 &     3.0 \\
    3 &  4.0 &     3.0 \\
    2 &  2.0 &     2.0 \\
    1 &  2.0 &     2.0 \\
\bottomrule
\end{tabular}



In [19]:
print(mle_women1.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     5.0 \\
    4 &  4.0 &     3.0 \\
    3 &  4.0 &     3.0 \\
    2 &  2.0 &     1.0 \\
    1 &  1.0 &     1.0 \\
\bottomrule
\end{tabular}



In [22]:
latest_estmates = pd.DataFrame({"Parameters": ["Lambda", "Delta"],
    "Point Estimates":[lambda_mle1, delta_mle1],
                               "Standard Errors":[se_lambda1, se_delta1]})
print(latest_estmates.to_latex(index = False))

\begin{tabular}{lrr}
\toprule
Parameters &  Point Estimates &  Standard Errors \\
\midrule
    Lambda &         0.758725 &         0.105999 \\
     Delta &         0.053191 &         0.011109 \\
\bottomrule
\end{tabular}



In [23]:
mle_args2 = [df2, mdf2, wdf2]
results2 = opt.minimize(criterion_function, 
                       params_initial, 
                       args=(mle_args2), 
                       method='L-BFGS-B', 
                       bounds=((0.01, 3), (0.01, 3)),
                       tol=1e-9,
                       options={'maxiter':1000})
results2

      fun: 3565.8405440796987
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0003638 , -0.00331966])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 48
      nit: 11
   status: 0
  success: True
        x: array([0.25354521, 0.05000558])

In [24]:
lambda_mle2, delta_mle2 = results2.x

In [25]:
vcv_mle2 = results2.hess_inv.todense()
se_lambda2 = np.sqrt(vcv_mle2[0,0])
se_delta2 = np.sqrt(vcv_mle2[1,1])

In [26]:
print("Lambda_hat is {}, with a standard error of {}.\n".format(lambda_mle2, se_lambda2))
print("Delta_hat is {}, with a standard error of {}.\n".format(delta_mle2, se_delta2))

Lambda_hat is 0.2535452112374287, with a standard error of 0.050765176996081664.

Delta_hat is 0.05000558290216397, with a standard error of 0.009274522092255032.



In [27]:
mle_men2, mle_women2 = generate_matchsets(mdf2, wdf2, [lambda_mle2, delta_mle2])

In [28]:
print(mle_men2.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     4.0 \\
    4 &  5.0 &     4.0 \\
    3 &  3.0 &     2.0 \\
    2 &  3.0 &     2.0 \\
    1 &  1.0 &     1.0 \\
\bottomrule
\end{tabular}



In [29]:
print(mle_women2.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     4.0 \\
    4 &  5.0 &     4.0 \\
    3 &  3.0 &     2.0 \\
    2 &  3.0 &     2.0 \\
    1 &  1.0 &     1.0 \\
\bottomrule
\end{tabular}



In [39]:
lb_lambda1 = lambda_mle1 - 1.96*se_lambda1
ub_lambda1 = lambda_mle1 + 1.96*se_lambda1
lb_lambda2 = lambda_mle2 - 1.96*se_lambda2
ub_lambda2 = lambda_mle2 + 1.96*se_lambda2
lb_delta1 = delta_mle1 - 1.96*se_delta1
ub_delta1 = delta_mle1 + 1.96*se_delta1
lb_delta2 = delta_mle2 - 1.96*se_delta2
ub_delta2 = delta_mle2 + 1.96*se_delta2



In [37]:
lb_lamnda1

0.5509668337019951

In [62]:
parDF = pd.DataFrame({"models": ["", "","", ""],
                      "variables":["lambda", "delta", "lambda", "delta"],
                      "Point Estimate":[lambda_mle1, delta_mle1, lambda_mle2, delta_mle2],
                      "S.E.":[se_lambda1,se_delta1,se_lambda2,se_delta2 ],
                      "Lower Bound (95%)": [lb_lambda1, lb_delta1,lb_lambda2, lb_delta2],
                      "Upper Bound (95%)":[ub_lambda1, ub_delta1,ub_lambda2, ub_delta2]
                     })
parDF

,models,variables,Point Estimate,S.E.,Lower Bound (95%),Upper Bound (95%)
0,,lambda,0.758725,0.105999,0.550967,0.966483
1,,delta,0.053191,0.011109,0.031418,0.074965
2,,lambda,0.253545,0.050765,0.154045,0.353045
3,,delta,0.050006,0.009275,0.031828,0.068184


In [63]:
print(parDF.round(3).to_latex(index = False))

\begin{tabular}{llrrrr}
\toprule
models & variables &  Point Estimate &   S.E. &  Lower Bound (95\%) &  Upper Bound (95\%) \\
\midrule
       &    lambda &           0.759 &  0.106 &              0.551 &              0.966 \\
       &     delta &           0.053 &  0.011 &              0.031 &              0.075 \\
       &    lambda &           0.254 &  0.051 &              0.154 &              0.353 \\
       &     delta &           0.050 &  0.009 &              0.032 &              0.068 \\
\bottomrule
\end{tabular}



In [48]:
mmatch_DF = pd.concat([mle_men1, mle_men2])
wmatch_DF =  pd.concat([mle_women1, mle_women2])

In [49]:
mmatch_DF

,type,max,reserv
0,5,5.0,5.0
1,4,4.0,3.0
2,3,4.0,3.0
3,2,2.0,2.0
4,1,2.0,2.0
0,5,5.0,4.0
1,4,5.0,4.0
2,3,3.0,2.0
3,2,3.0,2.0
4,1,1.0,1.0


In [50]:
wmatch_DF

,type,max,reserv
0,5,5.0,5.0
1,4,4.0,3.0
2,3,4.0,3.0
3,2,2.0,1.0
4,1,1.0,1.0
0,5,5.0,4.0
1,4,5.0,4.0
2,3,3.0,2.0
3,2,3.0,2.0
4,1,1.0,1.0


In [51]:
print(mmatch_DF.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     5.0 \\
    4 &  4.0 &     3.0 \\
    3 &  4.0 &     3.0 \\
    2 &  2.0 &     2.0 \\
    1 &  2.0 &     2.0 \\
    5 &  5.0 &     4.0 \\
    4 &  5.0 &     4.0 \\
    3 &  3.0 &     2.0 \\
    2 &  3.0 &     2.0 \\
    1 &  1.0 &     1.0 \\
\bottomrule
\end{tabular}



In [52]:
print(wmatch_DF.to_latex(index = False))

\begin{tabular}{rrr}
\toprule
 type &  max &  reserv \\
\midrule
    5 &  5.0 &     5.0 \\
    4 &  4.0 &     3.0 \\
    3 &  4.0 &     3.0 \\
    2 &  2.0 &     1.0 \\
    1 &  1.0 &     1.0 \\
    5 &  5.0 &     4.0 \\
    4 &  5.0 &     4.0 \\
    3 &  3.0 &     2.0 \\
    2 &  3.0 &     2.0 \\
    1 &  1.0 &     1.0 \\
\bottomrule
\end{tabular}



In [61]:
params_initial = np.array([.740, 0.002])
mle_args = [dfs[0], dfs[1], dfs[2]]
results_full = opt.minimize(criterion_function, 
                       params_initial, 
                       args=(mle_args), 
                       #method='L-BFGS-B', 
                       bounds=((0.001, 3), (0.001, 3)),
                       tol=1e-6,
                       options={'maxiter':1000})
results_full

      fun: 4929.940044992184
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 588.6060535 , 1452.75171235])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 168
      nit: 6
   status: 2
  success: False
        x: array([0.6422518, 0.0575303])

In [62]:
lambda_mle2, delta_mle2 = results_full.x

In [63]:
vcv_mle2 = results_full.hess_inv.todense()
se_lambda2 = np.sqrt(vcv_mle2[0,0])
se_delta2 = np.sqrt(vcv_mle2[1,1])

In [64]:
print("Lambda_hat is {}, with a standard error of {}.\n".format(lambda_mle2, se_lambda2))
print("Delta_hat is {}, with a standard error of {}.\n".format(delta_mle2, se_delta2))

Lambda_hat is 0.6422517951906717, with a standard error of 1.0.

Delta_hat is 0.05753030011285562, with a standard error of 1.0.

